In [ ]:
from mobile_sam import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from mobile_sam import SamAutomaticMaskGenerator
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
model_type = "vit_t"
sam_checkpoint = "./MobileSam/weights/mobile_sam.pt"

device = "cuda" if torch.cuda.is_available() else "cpu"

mobile_sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
mobile_sam.to(device=device)
mobile_sam.eval()

# predictor = SamPredictor(mobile_sam)
# predictor.set_image(<your_image>)
# masks, _, _ = predictor.predict(<input_prompts>)

In [ ]:
img_path = "./data/COD10K-v3/Train/Image/COD10K-CAM-1-Aquatic-1-BatFish-1.jpg"

image_bgr = cv2.imread(img_path)

# 2. Convert BGR -> RGB for SAM
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

# 3. Display original image
plt.figure(figsize=(8, 8))
plt.imshow(image_rgb)
plt.axis("off")
plt.title("Original Image")
plt.show()


In [ ]:
mask_generator = SamAutomaticMaskGenerator(mobile_sam)
masks = mask_generator.generate(image_rgb)

In [ ]:
len(masks)

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    img = np.ones((sorted_anns[0]['segmentation'].shape[0],
                   sorted_anns[0]['segmentation'].shape[1], 4))
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

plt.figure(figsize=(8, 8))
plt.imshow(image_rgb)
show_anns(masks)
plt.axis("off")
plt.title("Segmented Image with MobileSAM")
plt.show()

In [ ]:
# Load ground truth mask (white=object, black=background)
gt_mask = cv2.imread("./data/COD10K-v3/Train/GT_Object/COD10K-CAM-1-Aquatic-1-BatFish-1.png", cv2.IMREAD_GRAYSCALE)
gt_mask = (gt_mask > 127).astype(np.uint8)  # binarize (0 or 1)

def compute_iou(pred_mask, gt_mask):
    intersection = np.logical_and(pred_mask, gt_mask).sum()
    union = np.logical_or(pred_mask, gt_mask).sum()
    if union == 0:
        return 0.0
    return intersection / union

In [ ]:
best_mask = None
best_iou = -1

for m in masks:
    pred_mask = m['segmentation']
    iou = compute_iou(pred_mask, gt_mask)
    if iou > best_iou:
        best_iou = iou
        best_mask = pred_mask

print("Best IoU:", best_iou)

In [ ]:

plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.imshow(image_rgb)
plt.title("Original")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(gt_mask, cmap="gray")
plt.title("Ground Truth Mask")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(best_mask, cmap="gray")
plt.title(f"Best Predicted Mask (IoU={best_iou:.2f})")
plt.axis("off")

plt.show()